<h1 align="center"><font size="8">Capstone Final Project</font></h1>

<h1 align="center"><font size="5">Topic: Display a comparison of Suburbs in Sydney and Melbourne using Folium and </font></h1>

<h1> <font size="4">Introduction </font></h1>
<h1> <font size="3">Background: </font></h1> Melbourne and Sydney are the 2 main cities when living in Australia is considered or analysed. Although there are other beautiful and growing cities in Australia,People outside of Melbourne mainly look at the living standards and day to day life of these 2 cities mainly. Both the cities are similar in natural beauty, resource, places to visit, population, diversity, job opportunity and career opportunities. Yet an analytical view comparing the cities will have a very vast user base. My report will mainly target the migrant population to Australia

<h1> <font size="3">Problem: </font></h1> Australia has a very good migration policy and every year Australia accepts nearly 200,000 population as part of skilled migration programs. These people migrating to Australia normally do not have an analytical view of which City they have to land after receiving their permanent residency visa.Normally, these people talk to friends , colleagues and online forums to create an understanding of cities of Australia.

<h1> <font size="3">Interest: </font></h1> My analysis will use clustering methods to cluster the suburbs of both the city and display various characteristics of the suburbs in these cities. Readers will be able view data for schools, restaurants, playgrounds, places to visit and jobs created for major suburbs in these cities and create a perspective of both the city on their own. This analytical view will not give an idea of which city is better rather this report will only provide a visual analysis of the major suburbs of the city and leave it to the reader to create a perspective.


<h1> <font size="4">Data Acquisition </font></h1>
I would be mostly working on the below dataset and as I start developing all the analytical view would add any other required data:<br>
Suburb Coordinates: I am planning to do some web scraping to find the coordinates for Suburbs of Melbourne and use the kaggle dataset for Sydney suburbs : <a href="https://www.kaggle.com/koki25ando/city-list-of-australia" target="_blank">Sydney Suburb Coordinates</a>.<br>
Suburb Characteristics: Suburbs and it's nearby venue information will be fetched from foursquare places API.<br>
Melbourne Housing Data: Use the Kaggle dataset for :<a href="https://www.kaggle.com/anthonypino/melbourne-housing-marketMelbourne_housing_data" target="_blank">Melbourne housing data</a> <br>
Sydney housing data : Extract house pricing dataset from <a href="https://data.nsw.gov.au/data/dataset/housing-nsw-rent-and-sales-tables" target="_blank">NSW Government Sales Dataset</a><br>
Extract postcode of Sydney and Melbourne from Australia post API: <a href="http://v0.postcodeapi.com.au/" target="_blank">Search Suburb/postcode API</a>


<h1> <font size="4">Data Wrangling process Starts </font></h1>

Lets start wit Importing all the relevant packages

In [3]:
import pandas as pd

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import requests
from IPython.core.display import display, HTML
print('Libraries imported.')

Libraries imported.


<h1> <font size="2">Prepare Melbourne Data </font></h1>

Melourne Housing data has the required suburb, postcode and Co-ordinates. In the next few lines of codes, I will be extract the Melbourne Suburb data from Melbourne housing dataset


In [4]:
#using the read CSV method of pandas data frame
melbourne_df=pd.read_csv(r'C:\Users\Kalibrate-Biplab\Downloads\Melbourne-Housing-Data-Kaggle-master\Melbourne-Housing-Data-Kaggle-master\Data\Melbourne_housing_FULL.csv')
# Now I am extracting only those columns which are relavant as part of my anayslys
melbourneSuburb_df=melbourne_df[['Suburb','Postcode','Regionname','Lattitude','Longtitude']]
# Removing duplicates which exists as they were for different address and hourse sales.
melbourneSuburb_df=melbourneSuburb_df.drop_duplicates(subset = ['Suburb'])

melbourneSuburb_df.head()

,Suburb,Postcode,Regionname,Lattitude,Longtitude
0,Abbotsford,3067.0,Northern Metropolitan,-37.8014,144.9958
66,Airport West,3042.0,Western Metropolitan,-37.7180,144.8780
133,Albert Park,3206.0,Southern Metropolitan,-37.8459,144.9555
194,Alphington,3078.0,Northern Metropolitan,-37.7818,145.0198
230,Altona,3018.0,Western Metropolitan,-37.8700,144.8250


Now, As there a lot of suburbs in Melbourne and they are divided into different regions. As part of my analysis, I would be restricting this list only to Eastern Metropolitan region. The reason for doing so is limit the data which I will be fetching from foursquare as there is cap on the amount data i can fetch in a day from Foursquare. Also, It gives the analysis of displaying data only for a a part of the city. People normally select the region first before selecting the suburb when they plan to live in a suburb. This solution can be extended to let users choose a particular region from a UI field an run the rest of the analysis based on that.

In [5]:
#Filtering dataframe based on region and reset the index
melbourneSuburb_df=melbourneSuburb_df[melbourneSuburb_df['Regionname']=='Eastern Metropolitan']
melbourneSuburb_df=melbourneSuburb_df.reset_index(drop=True)
melbourneSuburb_df.head()

,Suburb,Postcode,Regionname,Lattitude,Longtitude
0,Box Hill,3128.0,Eastern Metropolitan,-37.8151,145.1350
1,Bulleen,3105.0,Eastern Metropolitan,-37.7599,145.0849
2,Doncaster,3108.0,Eastern Metropolitan,-37.7779,145.1270
3,Eaglemont,3084.0,Eastern Metropolitan,-37.7626,145.0570
4,Heidelberg Heights,3081.0,Eastern Metropolitan,-37.7501,145.0505


In [6]:
# Data was not clean and some suburbs didnt have co-ordinate data so removing the irrelevant NAN data
melbourneSuburb_df=melbourneSuburb_df.dropna()
melbourneSuburb_df.head()

,Suburb,Postcode,Regionname,Lattitude,Longtitude
0,Box Hill,3128.0,Eastern Metropolitan,-37.8151,145.1350
1,Bulleen,3105.0,Eastern Metropolitan,-37.7599,145.0849
2,Doncaster,3108.0,Eastern Metropolitan,-37.7779,145.1270
3,Eaglemont,3084.0,Eastern Metropolitan,-37.7626,145.0570
4,Heidelberg Heights,3081.0,Eastern Metropolitan,-37.7501,145.0505


<h1> <font size="2">Prepare Sydney Data </font></h1>

Sydney Suburb data will be prepared from 2 different ways. There is a Kaggle dataset for sydney suburbs( mentioned in the Data Acquisition). This dataset only has the Suburb name and Co-ordinates. So, I am using the Australia post API to fetch Suburb's Locality and Post code data. <br>
To limit the number of suburb from Sydney, I have got a list of Suburb from Greater Sydney Area and I would be using the same to filter the Suburb data for Sydney. Reason for limiting this is same as mentioned for Melbourne


In [8]:
Greater_Sydney_df=pd.read_excel(r'C:\Biplab\DataScience\Capstone\Capstone_Segmenting_clustering_Assignment\Data\GreaterSydney.xlsx')
sydney_suburbs_co_ordinates_df=pd.read_csv(r'C:\Biplab\DataScience\Capstone\Capstone_Segmenting_clustering_Assignment\Data\sydney_suburbs.csv')

In [11]:
Sydney_df = Greater_Sydney_df.join(sydney_suburbs_co_ordinates_df.set_index("Suburb"), on='Suburb').dropna()
Sydney_df.shape

(32, 3)

Creating a Method to Fetch Postcode and Locality Data from Australia post API

In [12]:
def getSuburbDetails(names):
    suburb_details_list=[]
    for name in (names):
        print("name:"+str(name))
            
        parameters={"name":name}
        response=requests.get("http://v0.postcodeapi.com.au/suburbs.json",params=parameters)
        data=response.json()
        print (response.content)
        for i in range (len(data)):
            print("anme from json:"+data[i]["name"]+" and name from df:"+name)
            if (data[i]["name"]==name and data[i]["state"]["abbreviation"]=='NSW'):
                print(data[i]["postcode"],data[i]["locality"])
                suburb_details_list.append([
                name,
                data[i]["postcode"],
                data[i]["locality"] ])
        
    suburb_details=pd.DataFrame([suburb_detail for  suburb_detail in suburb_details_list])
    suburb_details.columns=['Suburb','Postcode','Locality']
    return suburb_details

In [13]:
sydney_detailed_df=getSuburbDetails(Sydney_df["Suburb"])

name:Ashfield
b'[{"name": "Ashfield", "postcode": 1800, "state": {"name": "New South Wales", "abbreviation": "NSW"}, "locality": "ST GEORGE", "latitude": -33.883299999999998, "longitude": 151.11670000000001}, {"name": "Ashfield", "postcode": 2131, "state": {"name": "New South Wales", "abbreviation": "NSW"}, "locality": "ST GEORGE", "latitude": -33.883299999999998, "longitude": 151.11670000000001}, {"name": "Ashfield", "postcode": 4670, "state": {"name": "Queensland", "abbreviation": "QLD"}, "locality": "BRIS CITY CNTRY", "latitude": -24.854199999999999, "longitude": 152.23519999999999}, {"name": "Ashfield", "postcode": 6054, "state": {"name": "Western Australia", "abbreviation": "WA"}, "locality": "STIRLING", "latitude": -31.899000000000001, "longitude": 115.9451}]'
anme from json:Ashfield and name from df:Ashfield
1800 ST GEORGE
anme from json:Ashfield and name from df:Ashfield
2131 ST GEORGE
anme from json:Ashfield and name from df:Ashfield
anme from json:Ashfield and name from df:As

In [14]:
sydney_detailed_df.sort_values(by=['Postcode'])
sydney_detailed_df.drop_duplicates(subset=['Suburb'],inplace=True)
sydney_detailed_df=sydney_detailed_df.join(Sydney_df.set_index("Suburb"),on="Suburb")
sydney_detailed_df.head()

,Suburb,Postcode,Locality,Latitude,Longitude
0,Ashfield,1800,ST GEORGE,-33.889478,151.127412
2,Lane Cove,1595,CHATSWOOD,-33.816067,151.167820
4,Leichhardt,2040,LEICHHARDT,-33.881933,151.155867
5,Marrickville,1475,LEICHHARDT,-33.910385,151.155691
7,Mosman,2088,CHATSWOOD,-33.830135,151.244766


Renaming Melbourne DF columns to match with Sydney Df. It will be required later in the analysis during Merging of these Data frame

In [15]:
melbourneSuburb_df=melbourneSuburb_df.rename(columns={'Regionname':'Locality','Lattitude':'Latitude',
                                                      'Longtitude':'Longitude'})

In [16]:
melbourneSuburb_df.head()

,Suburb,Postcode,Locality,Latitude,Longitude
0,Box Hill,3128.0,Eastern Metropolitan,-37.8151,145.1350
1,Bulleen,3105.0,Eastern Metropolitan,-37.7599,145.0849
2,Doncaster,3108.0,Eastern Metropolitan,-37.7779,145.1270
3,Eaglemont,3084.0,Eastern Metropolitan,-37.7626,145.0570
4,Heidelberg Heights,3081.0,Eastern Metropolitan,-37.7501,145.0505


<h1> <font size="4">Lets do some Data visualization </font></h1>

Now, We have the initial Base datasets to be used in this analysis. Lets start with viewing the Sydney and Melbourne Suburbs on a Folium Map

Next Function will be used by both the cities to extract the City Co-ordinates.

In [17]:
def getCoordinates(address):

    geolocator = Nominatim(user_agent="Syd_mel_explorer")
    Location = geolocator.geocode(address)
    Latitude = Location.latitude
    Longitude = Location.longitude
    print('The geograpical coordinate of {} are {}, {}.'.format(address,Latitude, Longitude))
    return [Latitude,Longitude]

Below Function will create the map for a city 

In [30]:
def createMap(Suburb_df,latitude, longitude):
    map = folium.Map(location=[latitude, longitude], zoom_start=10)

    # add markers to map
    for lat, lng, borough, neighborhood in zip(Suburb_df['Latitude'], Suburb_df['Longitude'], Suburb_df['Locality'], Suburb_df['Suburb']):
        label = '{}, {}'.format(neighborhood, borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map)  

    return map

In [29]:
SydneyCoordintes=getCoordinates("Sydney")
#passing Hawthorn as address so that map display the eastern part of melbourne as the data we have chosen for Melbourne is eastern suburbs
MelbourneCoordintes=getCoordinates("Nunawading")


The geograpical coordinate of Sydney are -33.8548157, 151.2164539.
The geograpical coordinate of Nunawading are -37.8204496, 145.1752107.


In [31]:
SydneyMap=createMap(sydney_detailed_df,SydneyCoordintes[0],SydneyCoordintes[1])
MelbourneMap=createMap(melbourneSuburb_df,MelbourneCoordintes[0],MelbourneCoordintes[1])

Lets View the 2 maps together in 1 Frame:

In [32]:
htmlmap = HTML('<iframe srcdoc="{}" style="float:left; width: {}px; height: {}px; display:inline-block; width: 50%; margin: 0 auto; border: 2px solid black"></iframe>'
           '<iframe srcdoc="{}" style="float:right; width: {}px; height: {}px; display:inline-block; width: 50%; margin: 0 auto; border: 2px solid black"></iframe>'
           .format(SydneyMap.get_root().render().replace('"', '&quot;'),500,500,
                   MelbourneMap.get_root().render().replace('"', '&quot;'),500,500))
display(htmlmap)

<h1> <font size="2">Data Analsysis starts </font></h1>

Now, I will fetch FourSquare Venues data to do analysis for the suburbs of Sydney and Melbourn

Define the Fourquare properties to be used in the API calls

In [35]:
CLIENT_ID = 'LX3GSNOY1QGZ4VU1YZ5KAIX50VKGHWXYU53P0DQ4O5A2DGRH' # your Foursquare ID
CLIENT_SECRET = '4H54LVFYCYUJPHV5QUYZYGUXSDFPEUXKRXOMGBZ3CPKLKQY4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LX3GSNOY1QGZ4VU1YZ5KAIX50VKGHWXYU53P0DQ4O5A2DGRH
CLIENT_SECRET:4H54LVFYCYUJPHV5QUYZYGUXSDFPEUXKRXOMGBZ3CPKLKQY4


Next Function will be used by both the cities to fetch the venues data from fourquare API and store it in a Dataframe.

In [36]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Suburb', 
                  'Suburb Latitude', 
                  'Suburb Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [38]:
Melbourne_venues = getNearbyVenues(names=melbourneSuburb_df['Suburb'],
                                   latitudes=melbourneSuburb_df['Latitude'],
                                   longitudes=melbourneSuburb_df['Longitude']
                                  )

Box Hill
Bulleen
Doncaster
Eaglemont
Heidelberg Heights
Heidelberg West
Ivanhoe
Mont Albert
Templestowe Lower
Viewbank
Heidelberg
Bayswater
Bayswater North
Boronia
Briar Hill
Burwood East
Croydon
Croydon Hills
Croydon North
Doncaster East
Donvale
Eltham
Forest Hill
Glen Waverley
Heathmont
Lower Plenty
Mitcham
Montmorency
Ringwood
Ringwood East
Templestowe
The Basin
Vermont
Wantirna
Wantirna South
Blackburn North
Scoresby
Kilsyth
North Warrandyte
Croydon South
Knoxfield


In [39]:
Sydney_venues = getNearbyVenues(names=sydney_detailed_df['Suburb'],
                                   latitudes=sydney_detailed_df['Latitude'],
                                   longitudes=sydney_detailed_df['Longitude']
                                  )

Ashfield
Lane Cove
Leichhardt
Marrickville
Mosman
North Sydney
Randwick
Waverley
Woollahra
Auburn
Bankstown
Burwood
Canterbury
Canada Bay
Hunters Hill
Hurstville
Kogarah
Manly
Parramatta
Rockdale
Ryde
Strathfield
Willoughby
Blacktown
Camden
Campbelltown
Fairfield
Holroyd
Hornsby
Liverpool
Penrith
Sutherland


In [40]:
Melbourne_venues.head()

,Suburb,Suburb Latitude,Suburb Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Box Hill,-37.8151,145.135,Melbourne Cheer Academy,-37.812064,145.136083,Dance Studio
1,Box Hill,-37.8151,145.135,Top Life Fitness Club,-37.811750,145.136433,Gym / Fitness Center
2,Box Hill,-37.8151,145.135,The Tudor,-37.818845,145.134153,Hotel
3,Box Hill,-37.8151,145.135,Best Western Tudor Box Hill,-37.818906,145.134032,Hotel
4,Box Hill,-37.8151,145.135,Box Hill City Oval,-37.818741,145.136896,Cricket Ground


In [41]:
Sydney_venues.head()

,Suburb,Suburb Latitude,Suburb Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ashfield,-33.889478,151.127412,Manmaruya,-33.889603,151.126183,Japanese Restaurant
1,Ashfield,-33.889478,151.127412,Japanese Dining Bar by Shinsen,-33.889653,151.126254,Japanese Restaurant
2,Ashfield,-33.889478,151.127412,New Shanghai Night 新夜上海,-33.888461,151.124393,Shanghai Restaurant
3,Ashfield,-33.889478,151.127412,Choo Choo Train,-33.888275,151.124196,Chinese Restaurant
4,Ashfield,-33.889478,151.127412,Thai Number 1,-33.887142,151.125685,Thai Restaurant


Let's check the number of venues for each neighborhood calling pandas group by method.

In [42]:
Sydney_venues.groupby('Suburb').count()

,Suburb Latitude,Suburb Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Suburb,,,,,,
Ashfield,41,41,41,41,41,41
Auburn,3,3,3,3,3,3
Bankstown,50,50,50,50,50,50
Blacktown,25,25,25,25,25,25
Burwood,42,42,42,42,42,42
Camden,10,10,10,10,10,10
Campbelltown,4,4,4,4,4,4
Canada Bay,7,7,7,7,7,7
Canterbury,18,18,18,18,18,18


In [43]:
Melbourne_venues.groupby('Suburb').count()

,Suburb Latitude,Suburb Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Suburb,,,,,,
Bayswater,9,9,9,9,9,9
Bayswater North,5,5,5,5,5,5
Box Hill,6,6,6,6,6,6
Briar Hill,4,4,4,4,4,4
Bulleen,5,5,5,5,5,5
Burwood East,4,4,4,4,4,4
Croydon,1,1,1,1,1,1
Croydon North,2,2,2,2,2,2
Croydon South,4,4,4,4,4,4


Now lets begin analysis on the neighborhoods based on the venue category. I am using get_dummies method to create a new data frame indicating presence of each venue in the data frame. First I will analyse the Melbourne Venues and then Sydney

In [44]:
# one hot encoding
Melbourne_onehot = pd.get_dummies(Melbourne_venues[['Venue Category']], prefix="", prefix_sep="")
#del Melbourne_onehot['Suburb']
Melbourne_onehot['Suburb'] = Melbourne_venues['Suburb']
# move neighborhood column to the first column
fixed_columns = [Melbourne_onehot.columns[-1]] + list(Melbourne_onehot.columns[:-1])
Melbourne_onehot = Melbourne_onehot[fixed_columns]

Melbourne_onehot.head()

,Suburb,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Bakery,Bar,Bookstore,Brewery,Bus Station,Café,Cantonese Restaurant,Car Wash,Church,Coffee Shop,Construction & Landscaping,Convenience Store,Cricket Ground,Dance Studio,Deli / Bodega,Dim Sum Restaurant,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Football Stadium,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gay Bar,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Home Service,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Light Rail Station,Malay Restaurant,Mexican Restaurant,Mobile Phone Shop,National Park,Other Nightlife,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Pool,Portuguese Restaurant,Pub,Restaurant,Sandwich Place,Sculpture Garden,Shopping Mall,Skating Rink,South Indian Restaurant,Sporting Goods Shop,Sports Club,Supermarket,Sushi Restaurant,Tennis Court,Thai Restaurant,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar
0,Box Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Box Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Box Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Box Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Box Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Now, lets group the rows based on neighborhood and find the mean of frequency of occurence for each venue. This will help us find top venues for each neighborhood based on their frequency of occurance.

In [45]:
Melbourne_grouped = Melbourne_onehot.groupby('Suburb').mean().reset_index()
Melbourne_grouped.head()

,Suburb,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Bakery,Bar,Bookstore,Brewery,Bus Station,Café,Cantonese Restaurant,Car Wash,Church,Coffee Shop,Construction & Landscaping,Convenience Store,Cricket Ground,Dance Studio,Deli / Bodega,Dim Sum Restaurant,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Football Stadium,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gay Bar,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Home Service,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Light Rail Station,Malay Restaurant,Mexican Restaurant,Mobile Phone Shop,National Park,Other Nightlife,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Pool,Portuguese Restaurant,Pub,Restaurant,Sandwich Place,Sculpture Garden,Shopping Mall,Skating Rink,South Indian Restaurant,Sporting Goods Shop,Sports Club,Supermarket,Sushi Restaurant,Tennis Court,Thai Restaurant,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar
0,Bayswater,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.111111,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.111111,0.0,0.0,0.111111,0.222222,0.0
1,Bayswater North,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.2,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
2,Box Hill,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.166667,0.166667,0.00,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.166667,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
3,Briar Hill,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.25,0.0,0.000000,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.250000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
4,Bulleen,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.000000,0.2,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0


Now, I will step to create one hot dataframe and calculating Venues' mean of frequency for Syney

In [47]:
# one hot encoding
Sydney_onehot = pd.get_dummies(Sydney_venues[['Venue Category']], prefix="", prefix_sep="")
Sydney_onehot.head()

#del Sydney_onehot['Suburb']
Sydney_onehot['Suburb'] = Sydney_venues['Suburb']
# move neighborhood column to the first column
fixed_columns = [Sydney_onehot.columns[-1]] + list(Sydney_onehot.columns[:-1])
Sydney_onehot = Sydney_onehot[fixed_columns]

Sydney_onehot.head()

,Suburb,American Restaurant,Aquarium,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,Bar,Beach,Beijing Restaurant,Big Box Store,Bike Trail,Bistro,Bookstore,Bowling Alley,Bowling Green,Breakfast Spot,Bridal Shop,Buffet,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Camera Store,Cantonese Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cuban Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food & Drink Shop,Food Court,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,German Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,History Museum,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Moroccan Restaurant,Movie Theater,Multiplex,Music Store,Noodle House,Organic Grocery,Outdoor Supply Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shanghai Restaurant,Shopping Mall,Skating Rink,South American Restaurant,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Thrift / Vintage Store,Toy / Game Store,Train Station,Turkish Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Ashfield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ashfield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Ashfield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Ashfield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Ashfield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [48]:
Sydney_grouped = Sydney_onehot.groupby('Suburb').mean().reset_index()
Sydney_grouped.head()

,Suburb,American Restaurant,Aquarium,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,Bar,Beach,Beijing Restaurant,Big Box Store,Bike Trail,Bistro,Bookstore,Bowling Alley,Bowling Green,Breakfast Spot,Bridal Shop,Buffet,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Camera Store,Cantonese Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cuban Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food & Drink Shop,Food Court,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,German Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,History Museum,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Moroccan Restaurant,Movie Theater,Multiplex,Music Store,Noodle House,Organic Grocery,Outdoor Supply Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shanghai Restaurant,Shopping Mall,Skating Rink,South American Restaurant,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Thrift / Vintage Store,Toy / Game Store,Train Station,Turkish Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Ashfield,0.0,0.0,0.00,0.02439,0.02439,0.0,0.0,0.0,0.00,0.02439,0.0,0.02439,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.048780,0.0,0.00000,0.048780,0.00,0.00000,0.024390,0.0,0.02439,0.0,0.0,0.0,0.00,0.0,0.02439,0.0,0.02439,0.0,0.00000,0.00000,0.00000,0.073171,0.04878,0.02439,0.000000,0.000000,0.02439,0.00,0.0,0.0,0.00000,0.0,0.02439,0.0,0.0,0.0,0.0,0.0,0.0,0.02439,0.02439,0.00,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.04878,0.024390,0.00,0.0,0.00,0.02439,0.0,0.04878,0.0,0.0,0.0,0.00,0.00,0.00000,0.0,0.0,0.00000,0.00,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.02439,0.02439,0.04878,0.000000,0.0,0.0,0.00,0.02439,0.0,0.0,0.02439,0.00000,0.00000,0.0,0.0,0.04878,0.00000,0.0,0.0,0.0,0.00,0.00000,0.00,0.00,0.04878,0.02439,0.0,0.0,0.02439,0.0,0.0,0.0,0.00000,0.00,0.02439,0.0,0.0,0.00
1,Auburn,0.0,0.0,0.00,0.00000,0.00000,0.0,0.0,0.0,0.00,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.00,0.00000,0.000000,0.0,0.00000,0.0,0.0,0.0,0.00,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.333333,0.000000,0.00000,0.00,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.00,0.0,0.00,0.00000,0.0,0.00000,0.0,0.0,0.0,0.00,0.00,0.00000,0.0,0.0,0.00000,0.00,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.666667,0.0,0.0,0.00,0.00000,0.0,0.0,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.00,0.00000,0.00,0.00,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.00,0.00000,0.0,0.0,0.00
2,Bankstown,0.0,0.0,0.00,0.00000,0.00000,0.0,0.0,0.0,0.02,0.02000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.04,0.02,0.04,0.0,0.0,0.120000,0.0,0.00000,0.040000,0.02,0.00000,0.020000,0.0,0.02000,0.0,0.0,0.0,0.00,0.0,0.02000,0.0,0.00000,0.0,0.02000,0.02000,0.00000,0.000000,0.00000,0.00000,0.000000,0.040000,0.00000,0.02,0.0,0.0,0

Most of the time with the most common venues in a Suburb , users also want to know the average house price for each suburb. <br>
So, I will find a mean of hourse prices for each Melbourne Suburb using the Housing dataset I used during preparing the Melbourne Data. I also found a house sales dataset for Sydney suburbs from Kaggle( same has been mentioned in the data acquisition)
I will add the House Pricing for each suburbs of both the cities to give better perspectiveto the Suburb analysis.

#### All the house prices displayed are in 000s

In [49]:
melbourne_df=melbourne_df.dropna()
melbourne_df.set_index('Suburb')
melbourne_Suburb_price_df=melbourne_df[['Suburb','Price']]
melbourne_Suburb_price_df['Price']=melbourne_Suburb_price_df['Price']/1000
melbourne_Suburb_price_df.head()

C:\Python_files\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Suburb,Price
2,Abbotsford,1035.0
4,Abbotsford,1465.0
6,Abbotsford,1600.0
11,Abbotsford,1876.0
14,Abbotsford,1636.0


In [50]:
MelbournePrice_mean = melbourne_Suburb_price_df.groupby('Suburb').mean().reset_index()
MelbournePrice_mean.head()

,Suburb,Price
0,Abbotsford,1126.410256
1,Aberfeldie,1462.965517
2,Airport West,766.807712
3,Albanvale,540.333333
4,Albert Park,1950.985294


In [51]:
sydney_house_price=pd.read_excel(r'C:\Biplab\DataScience\Capstone\Capstone_Segmenting_clustering_Assignment\Data\Sydney_Housing_price.xlsx')
sydney_house_price.head()

,Suburb,House_Price
0,Ashfield,844.0
1,Botany Bay,1120.2
2,Lane Cove,885.4
3,Leichhardt,819.0
4,Marrickville,1224.1


In [52]:
# creating uniformity in column names as the same is required during merging of datasets
sydney_house_price=sydney_house_price.rename(columns={'House_Price':'Mean_house_price_in_K'})
MelbournePrice_mean=MelbournePrice_mean.rename(columns={'Price':'Mean_house_price_in_K'})

Now, Next phase of analysis I am going to work on merging the dataframes of Sydney and Melbour. I will merge the house prices into the mean calculatation for both cities. Then will merge both the city data into 1 dataframe and then run the data analysis which will display the top 10 common venues in each suburb and also the house prices in those suburbs.
The same merging will be used to applying clusering on the Suburb data so the clustering outcome is uniform for the cities.

Lets add this house price into the Mean calculated data frame as the same will play its part in clustering the suburbs.

In [53]:
Sydney_venue_price_merged = sydney_house_price

Sydney_venue_price_merged = Sydney_venue_price_merged.join(Sydney_grouped.set_index('Suburb'), on='Suburb')
Sydney_venue_price_merged=Sydney_venue_price_merged.dropna()
Sydney_venue_price_merged.head()

,Suburb,Mean_house_price_in_K,American Restaurant,Aquarium,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,Bar,Beach,Beijing Restaurant,Big Box Store,Bike Trail,Bistro,Bookstore,Bowling Alley,Bowling Green,Breakfast Spot,Bridal Shop,Buffet,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Camera Store,Cantonese Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cuban Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food & Drink Shop,Food Court,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,German Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,History Museum,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Moroccan Restaurant,Movie Theater,Multiplex,Music Store,Noodle House,Organic Grocery,Outdoor Supply Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shanghai Restaurant,Shopping Mall,Skating Rink,South American Restaurant,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Thrift / Vintage Store,Toy / Game Store,Train Station,Turkish Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Ashfield,844.0,0.0,0.0,0.0,0.02439,0.02439,0.0,0.000000,0.0,0.000000,0.024390,0.0,0.02439,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.048780,0.0,0.0,0.048780,0.0,0.000000,0.02439,0.0,0.02439,0.0,0.000000,0.0,0.0,0.00000,0.02439,0.0,0.02439,0.0,0.0,0.0,0.0,0.073171,0.048780,0.02439,0.000000,0.0,0.02439,0.0,0.0,0.000000,0.0,0.0,0.02439,0.0,0.0,0.0,0.0,0.0,0.0000,0.024390,0.024390,0.0,0.0,0.0,0.0,0.0000,0.000000,0.000000,0.0,0.0,0.000000,0.048780,0.02439,0.0,0.0,0.000000,0.02439,0.00000,0.04878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.024390,0.024390,0.04878,0.0,0.0,0.000000,0.0,0.024390,0.0,0.0,0.02439,0.0,0.00000,0.0,0.00000,0.04878,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.048780,0.024390,0.0,0.0,0.024390,0.0,0.0,0.0,0.0,0.0,0.02439,0.000000,0.0,0.0
2,Lane Cove,885.4,0.0,0.0,0.0,0.00000,0.00000,0.0,0.034483,0.0,0.068966,0.000000,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.137931,0.0,0.0,0.068966,0.0,0.000000,0.00000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.00000,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.0,0.00000,0.0,0.0,0.034483,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000,0.103448,0.0,0.0,0.0,0.0,0.0000,0.034483,0.000000,0.0,0.0,0.000000,0.068966,0.00000,0.0,0.0,0.000000,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.034483,0.0,0.0,0.000000,0.0,0.034483,0.0,0.034483,0.103448,0.00000,0.0,0.0,0.034483,0.0,0.034483,0.0,0.0,0.00000,0.0,0.00000,0.0,0.00000,0.00000,0.034483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.034483,0.034483,0.0,0.0,0.034483,0.0,0.0,0.0,0.0,0.0,0.00000,0.034483,0.0,0.0
3,Leichhardt,819.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.000000,0.0,0.028571,0.028571,0.0,0.00000,0.0,0.0,0.0,0.028571,0.0,0.0,0.0,0.0,0.0,0.028571,0.0,0.0,0.0,0.200000,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.00000,0.00000,0.0,0.00000

In [54]:
MelbournePrice_mean_merged = MelbournePrice_mean

MelbournePrice_mean_merged = MelbournePrice_mean_merged.join(Melbourne_grouped.set_index('Suburb'), on='Suburb')
MelbournePrice_mean_merged=MelbournePrice_mean_merged.dropna().reset_index(drop=True)
MelbournePrice_mean_merged.head()

,Suburb,Mean_house_price_in_K,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Bakery,Bar,Bookstore,Brewery,Bus Station,Café,Cantonese Restaurant,Car Wash,Church,Coffee Shop,Construction & Landscaping,Convenience Store,Cricket Ground,Dance Studio,Deli / Bodega,Dim Sum Restaurant,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Football Stadium,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gay Bar,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Home Service,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Light Rail Station,Malay Restaurant,Mexican Restaurant,Mobile Phone Shop,National Park,Other Nightlife,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Pool,Portuguese Restaurant,Pub,Restaurant,Sandwich Place,Sculpture Garden,Shopping Mall,Skating Rink,South Indian Restaurant,Sporting Goods Shop,Sports Club,Supermarket,Sushi Restaurant,Tennis Court,Thai Restaurant,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar
0,Bayswater,766.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.111111,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.111111,0.0,0.0,0.111111,0.222222,0.0
1,Bayswater North,750.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.2,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
2,Box Hill,1495.067294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.166667,0.166667,0.00,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.166667,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
3,Briar Hill,808.214286,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.25,0.0,0.000000,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.250000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
4,Bulleen,1150.754054,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.000000,0.2,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0


In [55]:
# adding the city column which will be later used to segregate the data to view on the map
Sydney_venue_price_merged.insert(0, 'City', 'Sydney')
MelbournePrice_mean_merged.insert(0, 'City', 'Melbourne')

Lets concat both the dataframes for Suburb Clustering and Venue Analysis

In [56]:
frames = [MelbournePrice_mean_merged, Sydney_venue_price_merged]
mel_syd_merged=pd.concat(frames, sort=False)
mel_syd_merged.head()

,City,Suburb,Mean_house_price_in_K,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Bakery,Bar,Bookstore,Brewery,Bus Station,Café,Cantonese Restaurant,Car Wash,Church,Coffee Shop,Construction & Landscaping,Convenience Store,Cricket Ground,Dance Studio,Deli / Bodega,Dim Sum Restaurant,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Football Stadium,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gay Bar,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Home Service,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Light Rail Station,Malay Restaurant,Mexican Restaurant,Mobile Phone Shop,National Park,Other Nightlife,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Pool,Portuguese Restaurant,Pub,Restaurant,Sandwich Place,Sculpture Garden,Shopping Mall,Skating Rink,South Indian Restaurant,Sporting Goods Shop,Sports Club,Supermarket,Sushi Restaurant,Tennis Court,Thai Restaurant,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,American Restaurant,Aquarium,Arts & Crafts Store,BBQ Joint,Beach,Beijing Restaurant,Big Box Store,Bike Trail,Bistro,Bowling Alley,Bowling Green,Breakfast Spot,Bridal Shop,Buffet,Burger Joint,Bus Stop,Business Service,Camera Store,Chinese Restaurant,Chocolate Shop,Clothing Store,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Drugstore,Falafel Restaurant,Farmers Market,Food & Drink Shop,Food Court,Fried Chicken Joint,Frozen Yogurt Shop,Gastropub,German Restaurant,Golf Course,Gourmet Shop,History Museum,Hookah Bar,Indie Movie Theater,Indonesian Restaurant,Intersection,Juice Bar,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Middle Eastern Restaurant,Moroccan Restaurant,Movie Theater,Multiplex,Music Store,Noodle House,Organic Grocery,Outdoor Supply Store,Paper / Office Supplies Store,Persian Restaurant,Platform,Plaza,Ramen Restaurant,Record Shop,Salad Place,Scenic Lookout,Seafood Restaurant,Shanghai Restaurant,South American Restaurant,Spanish Restaurant,Sports Bar,Steakhouse,Tapas Restaurant,Tea Room,Thrift / Vintage Store,Toy / Game Store,Turkish Restaurant,Wine Bar,Wine Shop,Women's Store
0,Melbourne,Bayswater,766.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.111111,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.111111,0.0,0.0,0.111111,0.222222,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Melbourne,Bayswater North,750.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.2,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Melbourne,Box Hill,1495.067294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.16

In [57]:
mel_syd_merged=mel_syd_merged.fillna(0.0)
mel_syd_merged.head()

,City,Suburb,Mean_house_price_in_K,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Bakery,Bar,Bookstore,Brewery,Bus Station,Café,Cantonese Restaurant,Car Wash,Church,Coffee Shop,Construction & Landscaping,Convenience Store,Cricket Ground,Dance Studio,Deli / Bodega,Dim Sum Restaurant,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Football Stadium,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gay Bar,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Home Service,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Light Rail Station,Malay Restaurant,Mexican Restaurant,Mobile Phone Shop,National Park,Other Nightlife,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Pool,Portuguese Restaurant,Pub,Restaurant,Sandwich Place,Sculpture Garden,Shopping Mall,Skating Rink,South Indian Restaurant,Sporting Goods Shop,Sports Club,Supermarket,Sushi Restaurant,Tennis Court,Thai Restaurant,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,American Restaurant,Aquarium,Arts & Crafts Store,BBQ Joint,Beach,Beijing Restaurant,Big Box Store,Bike Trail,Bistro,Bowling Alley,Bowling Green,Breakfast Spot,Bridal Shop,Buffet,Burger Joint,Bus Stop,Business Service,Camera Store,Chinese Restaurant,Chocolate Shop,Clothing Store,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Drugstore,Falafel Restaurant,Farmers Market,Food & Drink Shop,Food Court,Fried Chicken Joint,Frozen Yogurt Shop,Gastropub,German Restaurant,Golf Course,Gourmet Shop,History Museum,Hookah Bar,Indie Movie Theater,Indonesian Restaurant,Intersection,Juice Bar,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Middle Eastern Restaurant,Moroccan Restaurant,Movie Theater,Multiplex,Music Store,Noodle House,Organic Grocery,Outdoor Supply Store,Paper / Office Supplies Store,Persian Restaurant,Platform,Plaza,Ramen Restaurant,Record Shop,Salad Place,Scenic Lookout,Seafood Restaurant,Shanghai Restaurant,South American Restaurant,Spanish Restaurant,Sports Bar,Steakhouse,Tapas Restaurant,Tea Room,Thrift / Vintage Store,Toy / Game Store,Turkish Restaurant,Wine Bar,Wine Shop,Women's Store
0,Melbourne,Bayswater,766.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.111111,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.111111,0.0,0.0,0.111111,0.222222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Melbourne,Bayswater North,750.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.2,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Melbourne,Box Hill,1495.067294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.16

Next function ised used to find the top n venues for each suburb

In [58]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now the following display an analsys of top 10 most common venues of each suburb and the mean housing prices for those suburs

In [59]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City','Suburb','Mean_house_price_in_K']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
syd_mel_Suburb_venues_sorted = pd.DataFrame(columns=columns)
syd_mel_Suburb_venues_sorted['Suburb'] = mel_syd_merged['Suburb']
syd_mel_Suburb_venues_sorted['City'] = mel_syd_merged['City']
syd_mel_Suburb_venues_sorted['Mean_house_price_in_K'] = mel_syd_merged['Mean_house_price_in_K']

for ind in np.arange(mel_syd_merged.shape[0]):
    syd_mel_Suburb_venues_sorted.iloc[ind, 3:] = return_most_common_venues(mel_syd_merged.iloc[ind, :], num_top_venues)

syd_mel_Suburb_venues_sorted.head()

,City,Suburb,Mean_house_price_in_K,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Melbourne,Bayswater,766.033333,Vietnamese Restaurant,Supermarket,Malay Restaurant,Cantonese Restaurant,Grocery Store,Thai Restaurant,Dumpling Restaurant,Video Store,Park,Pool
1,Melbourne,Bayswater North,750.500000,Ice Cream Shop,Convenience Store,Pizza Place,Sporting Goods Shop,Gas Station,Women's Store,Performing Arts Venue,Pool,Playground,Pharmacy
2,Melbourne,Box Hill,1495.067294,Hotel,Dance Studio,Cricket Ground,Gym / Fitness Center,Grocery Store,Women's Store,Performing Arts Venue,Pool,Playground,Pizza Place
3,Melbourne,Briar Hill,808.214286,Deli / Bodega,Gym / Fitness Center,Electronics Store,Restaurant,Light Rail Station,Pet Store,Portuguese Restaurant,Italian Restaurant,Pool,Playground
4,Melbourne,Bulleen,1150.754054,Park,Café,Car Wash,Sandwich Place,Art Gallery,Japanese Restaurant,Pub,Portuguese Restaurant,Pool,Playground


In [60]:
syd_mel_Suburb_venues_sorted.tail()

,City,Suburb,Mean_house_price_in_K,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,Sydney,Holroyd,469.6,Men's Store,Intersection,Gym,Park,History Museum,Women's Store,Pizza Place,Pharmacy,Pet Store,Performing Arts Venue
35,Sydney,Hornsby,585.0,Paper / Office Supplies Store,Café,Pool,Japanese Restaurant,Thrift / Vintage Store,Platform,Bowling Alley,Pet Store,Park,Playground
36,Sydney,Liverpool,550.3,Café,Coffee Shop,Fast Food Restaurant,Shopping Mall,Supermarket,Sandwich Place,Seafood Restaurant,Gym,Department Store,Juice Bar
37,Sydney,Penrith,599.6,Café,Fast Food Restaurant,Department Store,Thai Restaurant,Performing Arts Venue,Electronics Store,Burger Joint,Sandwich Place,Pub,Dessert Shop
39,Sydney,Sutherland,973.0,Café,Pub,Thai Restaurant,Sandwich Place,Park,Gym,Farmers Market,Grocery Store,Ramen Restaurant,Pizza Place


I will use the applied data science knowledge to run Kmeans clusterting on the Merged data frams for Sydney and Melbourne and cluster the suburb into different clusters. This help users to view similar kind of Suburbs. The solution can be improvised to add a user action to choose a cluster and view its details.

In [72]:
# run k-means clustering. I am creating 6 clusters looking at the amount of data 
kclusters=6

In [62]:
Syd_mel_clustering = mel_syd_merged.drop(['City','Suburb'], 1)

# run k-means clustering. I am creating 6 clusters looking at the amount of data 
syd_mel_kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Syd_mel_clustering)

# check cluster labels generated for each row in the dataframe
syd_mel_kmeans.labels_[0:20] 

array([5, 5, 1, 0, 3, 3, 0, 0, 0, 1, 1, 4, 3, 3, 1, 3, 0, 5, 5, 1])

Now, Lets merge the Whole Sydney and Melbourne data with the Analysed venues and Clsuter data sets.

In [63]:
frames = [sydney_detailed_df, melbourneSuburb_df]
mel_syd__main_merged=pd.concat(frames, sort=False)
mel_syd__main_merged.head()

,Suburb,Postcode,Locality,Latitude,Longitude
0,Ashfield,1800.0,ST GEORGE,-33.889478,151.127412
2,Lane Cove,1595.0,CHATSWOOD,-33.816067,151.167820
4,Leichhardt,2040.0,LEICHHARDT,-33.881933,151.155867
5,Marrickville,1475.0,LEICHHARDT,-33.910385,151.155691
7,Mosman,2088.0,CHATSWOOD,-33.830135,151.244766


First add the cluster label to the Data frame of the venues analysis and then Merge it with main datasets

In [64]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
#del syd_mel_Suburb_venues_sorted['Cluster Labels']
syd_mel_Suburb_venues_sorted.insert(0, 'Cluster Labels', syd_mel_kmeans.labels_)
# need to merge the super dfs and then add the clusters
Syd_mel_final_merged = mel_syd__main_merged

# merge venues analyses, cluster analysis and Main super merged dataframes 
Syd_mel_final_merged = Syd_mel_final_merged.join(syd_mel_Suburb_venues_sorted.set_index('Suburb'), on='Suburb')
Syd_mel_final_merged=Syd_mel_final_merged.dropna()
Syd_mel_final_merged['Cluster Labels']=Syd_mel_final_merged['Cluster Labels'].astype(int)
Syd_mel_final_merged.head() # check the last columns!

,Suburb,Postcode,Locality,Latitude,Longitude,Cluster Labels,City,Mean_house_price_in_K,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ashfield,1800.0,ST GEORGE,-33.889478,151.127412,0,Sydney,844.0,Dumpling Restaurant,Electronics Store,Supermarket,Japanese Restaurant,Chinese Restaurant,Platform,Café,Shanghai Restaurant,Malay Restaurant,Asian Restaurant
2,Lane Cove,1595.0,CHATSWOOD,-33.816067,151.167820,0,Sydney,885.4,Café,Gym,Pizza Place,Chinese Restaurant,Japanese Restaurant,Bakery,Fried Chicken Joint,Pharmacy,Pool,Pub
4,Leichhardt,2040.0,LEICHHARDT,-33.881933,151.155867,0,Sydney,819.0,Italian Restaurant,Café,Japanese Restaurant,Pub,Thai Restaurant,Burger Joint,Farmers Market,Gym,Lebanese Restaurant,Grocery Store
5,Marrickville,1475.0,LEICHHARDT,-33.910385,151.155691,1,Sydney,1224.1,Vietnamese Restaurant,Café,Ice Cream Shop,Greek Restaurant,Bar,Gym,Coffee Shop,Grocery Store,Juice Bar,Pizza Place
7,Mosman,2088.0,CHATSWOOD,-33.830135,151.244766,1,Sydney,1395.7,Café,Bakery,Pub,Clothing Store,Japanese Restaurant,Fried Chicken Joint,Grocery Store,Creperie,Burger Joint,Italian Restaurant


Above analsis can be used to view data for different clueters by running filters based on Cluster Label. Following is one example where I am viewing all the suburbs analysed data in cluster 5.

In [65]:
Syd_mel_final_merged.loc[Syd_mel_final_merged['Cluster Labels'] == 5, Syd_mel_final_merged.columns[[1] + list(range(4, Syd_mel_final_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,City,Mean_house_price_in_K,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,2193.0,151.117015,5,Sydney,703.700000,Park,Platform,Café,Supermarket,Camera Store,Skating Rink,Fast Food Restaurant,Korean Restaurant,Pub,Middle Eastern Restaurant
21,2046.0,151.115155,5,Sydney,696.300000,Outdoor Supply Store,Sports Club,Golf Course,Athletics & Sports,Playground,Gym,Coffee Shop,Pool,Pizza Place,Pharmacy
23,2220.0,151.100361,5,Sydney,702.600000,Aquarium,Cricket Ground,Women's Store,Other Nightlife,Pool,Playground,Pizza Place,Pharmacy,Pet Store,Performing Arts Venue
34,2135.0,151.081176,5,Sydney,687.200000,Business Service,Women's Store,Pub,Pool,Playground,Pizza Place,Pharmacy,Pet Store,Performing Arts Venue,Park
35,2068.0,151.200278,5,Sydney,758.200000,Café,Pizza Place,Japanese Restaurant,Bus Station,Malay Restaurant,Gym / Fitness Center,Gas Station,Furniture / Home Store,Fish & Chips Shop,Pub
37,2570.0,150.696170,5,Sydney,671.300000,Supermarket,Café,Park,Bar,Pharmacy,Coffee Shop,Fast Food Restaurant,American Restaurant,Mobile Phone Shop,Mexican Restaurant
4,3081.0,145.050500,5,Melbourne,759.767857,Café,IT Services,Gas Station,Fast Food Restaurant,Pool,Playground,Pizza Place,Pharmacy,Pet Store,Performing Arts Venue
5,3081.0,145.039700,5,Melbourne,645.583333,Sculpture Garden,Playground,Pizza Place,Gym,Mobile Phone Shop,Women's Store,Park,Pool,Pharmacy,Pet Store
14,3153.0,145.260750,5,Melbourne,766.033333,Vietnamese Restaurant,Supermarket,Malay Restaurant,Cantonese Restaurant,Grocery Store,Thai Restaurant,Dumpling Restaurant,Video Store,Park,Pool
15,3153.0,145.274880,5,Melbourne,750.500000,Ice Cream Shop,Convenience Store,Pizza Place,Sporting Goods Shop,Gas Station,Women's Store,Performing Arts Venue,Pool,Playground,Pharmacy


####  Final Data visualization of the analysed data into a map

First Step is to Create a function which will generate clustered map for both the cities

In [75]:
# create map
def Create_clusterMap(merged_df,latitude, longitude):
    # create map
    map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

    # set color scheme for the clusters
    x = np.arange(kclusters)
    ys = [i + x + (i*x)**2 for i in range(kclusters)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    # add markers to the map
    markers_colors = []
    for lat, lon, poi, cluster in zip(merged_df['Latitude'], merged_df['Longitude'], merged_df['Suburb'], merged_df['Cluster Labels']):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)

    return map_clusters

Now, to display both the cities in 2 different map view, I am going to segregate the final analysed clustered data frames into the 2 city data frames.

In [69]:
Mel_cluster_map_df=Syd_mel_final_merged[Syd_mel_final_merged['City']=='Melbourne']
Mel_cluster_map_df.head()


,Suburb,Postcode,Locality,Latitude,Longitude,Cluster Labels,City,Mean_house_price_in_K,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Box Hill,3128.0,Eastern Metropolitan,-37.8151,145.1350,1,Melbourne,1495.067294,Hotel,Dance Studio,Cricket Ground,Gym / Fitness Center,Grocery Store,Women's Store,Performing Arts Venue,Pool,Playground,Pizza Place
1,Bulleen,3105.0,Eastern Metropolitan,-37.7599,145.0849,3,Melbourne,1150.754054,Park,Café,Car Wash,Sandwich Place,Art Gallery,Japanese Restaurant,Pub,Portuguese Restaurant,Pool,Playground
2,Doncaster,3108.0,Eastern Metropolitan,-37.7779,145.1270,1,Melbourne,1325.555072,Bus Station,Sushi Restaurant,Gym / Fitness Center,Garden,Pool,Playground,Pizza Place,Pharmacy,Pet Store,Performing Arts Venue
3,Eaglemont,3084.0,Eastern Metropolitan,-37.7626,145.0570,4,Melbourne,1615.222222,Café,Grocery Store,Train Station,Park,Women's Store,Pool,Playground,Pizza Place,Pharmacy,Pet Store
4,Heidelberg Heights,3081.0,Eastern Metropolitan,-37.7501,145.0505,5,Melbourne,759.767857,Café,IT Services,Gas Station,Fast Food Restaurant,Pool,Playground,Pizza Place,Pharmacy,Pet Store,Performing Arts Venue


In [70]:
Syd_cluster_map_df=Syd_mel_final_merged[Syd_mel_final_merged['City']=='Sydney']
Syd_cluster_map_df.head()

,Suburb,Postcode,Locality,Latitude,Longitude,Cluster Labels,City,Mean_house_price_in_K,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ashfield,1800.0,ST GEORGE,-33.889478,151.127412,0,Sydney,844.0,Dumpling Restaurant,Electronics Store,Supermarket,Japanese Restaurant,Chinese Restaurant,Platform,Café,Shanghai Restaurant,Malay Restaurant,Asian Restaurant
2,Lane Cove,1595.0,CHATSWOOD,-33.816067,151.167820,0,Sydney,885.4,Café,Gym,Pizza Place,Chinese Restaurant,Japanese Restaurant,Bakery,Fried Chicken Joint,Pharmacy,Pool,Pub
4,Leichhardt,2040.0,LEICHHARDT,-33.881933,151.155867,0,Sydney,819.0,Italian Restaurant,Café,Japanese Restaurant,Pub,Thai Restaurant,Burger Joint,Farmers Market,Gym,Lebanese Restaurant,Grocery Store
5,Marrickville,1475.0,LEICHHARDT,-33.910385,151.155691,1,Sydney,1224.1,Vietnamese Restaurant,Café,Ice Cream Shop,Greek Restaurant,Bar,Gym,Coffee Shop,Grocery Store,Juice Bar,Pizza Place
7,Mosman,2088.0,CHATSWOOD,-33.830135,151.244766,1,Sydney,1395.7,Café,Bakery,Pub,Clothing Store,Japanese Restaurant,Fried Chicken Joint,Grocery Store,Creperie,Burger Joint,Italian Restaurant


In [76]:
Sydney_cluster_map=Create_clusterMap(Syd_cluster_map_df,SydneyCoordintes[0],SydneyCoordintes[1])
Melbourne_cluste_map=Create_clusterMap(Mel_cluster_map_df,MelbourneCoordintes[0],MelbourneCoordintes[1])

In [77]:
htmlmap = HTML('<iframe srcdoc="{}" style="float:left; width: {}px; height: {}px; display:inline-block; width: 50%; margin: 0 auto; border: 2px solid black"></iframe>'
           '<iframe srcdoc="{}" style="float:right; width: {}px; height: {}px; display:inline-block; width: 50%; margin: 0 auto; border: 2px solid black"></iframe>'
           .format(Sydney_cluster_map.get_root().render().replace('"', '&quot;'),500,500,
                   Melbourne_cluste_map.get_root().render().replace('"', '&quot;'),500,500))
display(htmlmap)